In [1]:
from bs4 import BeautifulSoup
import requests
page = requests.get("https://www.nyrp.org/green-spaces/list#")
soup = BeautifulSoup(page.content, 'html.parser')

In [2]:
data = []
for a in soup.find_all('a'):
    coords = a.get('data-coordinates')
    if coords: # only want those that aren't null
        entry = {'coords': coords,
                 'lat': coords.split(',')[0],
                 'long': coords.split(',')[1],
                 'address': a.get('data-address')}
        data.append(entry)

In [3]:
import pandas as pd
data = pd.DataFrame(data)
data.head()

,address,coords,lat,long
0,735 East 211th St,"40.877499,-73.863489",40.877499,-73.863489
1,1818 Bathgate Ave,"40.845051,-73.897747",40.845051,-73.897747
2,1017 Teller Ave,"40.82815,-73.914356",40.82815,-73.914356
3,1328 Clay Ave,"40.834934,-73.908892",40.834934,-73.908892
4,2044 Prospect Ave,"40.845818,-73.887851",40.845818,-73.887851


In [4]:
# hardcoding boroughs, forgive me :(
data['borough'] = ['Bronx'] * 13 + ['Brooklyn'] * 20 + ['Manhatten'] * 18 + ['Queens'] * 4 + ['Staten Island'] * 1
data.head()

,address,coords,lat,long,borough
0,735 East 211th St,"40.877499,-73.863489",40.877499,-73.863489,Bronx
1,1818 Bathgate Ave,"40.845051,-73.897747",40.845051,-73.897747,Bronx
2,1017 Teller Ave,"40.82815,-73.914356",40.82815,-73.914356,Bronx
3,1328 Clay Ave,"40.834934,-73.908892",40.834934,-73.908892,Bronx
4,2044 Prospect Ave,"40.845818,-73.887851",40.845818,-73.887851,Bronx


In [5]:
data.to_csv('NYRP_gardens2.csv')

In [6]:
# Setting up modules
from geopandas import GeoDataFrame
import numpy as np
import pandas as pd
from shapely.geometry import Point
import matplotlib.pylab as plt
import geopy

In [17]:
data['long'] = pd.to_numeric(data['long'])
data['lat'] = pd.to_numeric(data['lat'])

In [34]:
data['geometry'] = data.apply(lambda data: Point((data.long, data.lat)), axis=1)
gdf = GeoDataFrame(data, geometry = 'geometry')

In [27]:
import geopandas as gpd
nta = gpd.read_file('data/Neighborhood Tabulation Areas.geojson')

In [28]:
nta.head()

,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,BK88,54005018.7472,047,Borough Park,39247.2280737,Brooklyn,3,(POLYGON ((-73.97604935657381 40.6312759056467...
1,QN51,52488276.477,081,Murray Hill,33266.904811,Queens,4,(POLYGON ((-73.80379022888246 40.7756101117924...
2,QN27,19726845.26,081,East Elmhurst,19816.7115378,Queens,4,(POLYGON ((-73.86109724401859 40.7636644770877...
3,QN07,22887772.7683,081,Hollis,20976.3358374,Queens,4,(POLYGON ((-73.75725671509139 40.7181386016625...
4,MN06,10647077.6122,061,Manhattanville,17040.6865482,Manhattan,1,(POLYGON ((-73.94607828608069 40.8212632160616...


In [37]:
gdf2 = gpd.sjoin(gdf, nta, op="intersects")

In [39]:
gdf2.head()

,address,coords,lat,long,borough,geometry,index_right,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code
0,735 East 211th St,"40.877499,-73.863489",40.877499,-73.863489,Bronx,POINT (-73.863489 40.877499),75,BX44,36273595.3423,005,Williamsbridge-Olinville,27351.0773787,Bronx,2
1,1818 Bathgate Ave,"40.845051,-73.897747",40.845051,-73.897747,Bronx,POINT (-73.897747 40.845051),128,BX01,16451615.9732,005,Claremont-Bathgate,29972.7777202,Bronx,2
2,1017 Teller Ave,"40.82815,-73.914356",40.828150,-73.914356,Bronx,POINT (-73.91435600000001 40.82815),175,BX14,18222401.7791,005,East Concourse-Concourse Village,27223.8477217,Bronx,2
3,1328 Clay Ave,"40.834934,-73.908892",40.834934,-73.908892,Bronx,POINT (-73.90889200000001 40.834934),175,BX14,18222401.7791,005,East Concourse-Concourse Village,27223.8477217,Bronx,2
4,2044 Prospect Ave,"40.845818,-73.887851",40.845818,-73.887851,Bronx,POINT (-73.887851 40.845818),144,BX17,19255162.9005,005,East Tremont,29633.136729,Bronx,2
